## Summary of Changes (Jul 2024)

- Instead of averging out the replicates - you now input a dataframe with columns "gene, timepoint, replicate"
  - The code takes the data and performs min-max normalisation *by replicate*.
  - It then merges the dataframes for parent and child gene by both timepoint and replicate
  - Hence, instead of giving it 10 datapoints for 10 timepoints - it now takes 30 datapoints (if you have 3 replicates for each timepoint)
- Added an option for "Shift timepoints".
  - instead of comparing expression levels for timepoint t between parent and child, you have the option to check for if parents expression at t explains t+1 expression for the child
  - set it True or False
  - Might make network inference more 'logical'.
  
- Change in line 115 in GRN functions.py - minor. (Consequence - Cannot do 2 parents against 1 child simultaneously, only 1 to 1 interactions without making changes to the code below.)
  
##### Important: Make sure you have equal number of replicates for each gene. (Don't mix and match genes from different dataset - it will result in NaNs that you will have to take care of yourself :))

In [1]:
from GRN_functions import *

import pyreadr 
import networkx as nx

In [6]:
# Read the dummy data
expression_data = pd.read_csv("dummy_data_format.csv")


parent_genes = ["G1", "G2"]
child_genes = ["G3"]
shift_timepoints = True
parent_child_gene_dict = {}

def normalise_expression_data_by_replicates(expression_df: pd.DataFrame) -> pd.DataFrame:
    replicates = expression_df["replicate"].unique()
    pivoted_expression_data = expression_df.pivot_table(values="tpm", index="timepoint", columns="replicate")
    for replicate in replicates:
        pivoted_expression_data[replicate] = (pivoted_expression_data[replicate] - min(pivoted_expression_data[replicate]))/(max(pivoted_expression_data[replicate]) - min(pivoted_expression_data[replicate]))
    return pd.melt(pivoted_expression_data.reset_index(), id_vars="timepoint", value_name="tpm")

for parent_gene in parent_genes:
    expression_data_parent = expression_data[expression_data["gene"] == parent_gene][["tpm", "replicate", "timepoint"]].copy(deep=True).reset_index(drop=True)
    expression_data_parent = normalise_expression_data_by_replicates(expression_data_parent)

    for child_gene in child_genes:
        expression_data_child = expression_data[expression_data["gene"] == child_gene][["tpm", "replicate", "timepoint"]].copy(deep=True).reset_index(drop=True)
        expression_data_child = normalise_expression_data_by_replicates(expression_data_child)

        if shift_timepoints:
            timepoints = expression_data_child["timepoint"].unique()
            # remove the first timepoint 
            expression_data_child = expression_data_child[expression_data_child["timepoint"] > timepoints[0]]
            # replace each timepoint with its previous timepoint
            expression_data_child["timepoint"].replace({timepoint: timepoints[i-1] for i, timepoint in enumerate(timepoints) if i > 0}, inplace=True)

        expression_data_parent_child = pd.merge(left=expression_data_parent,right=expression_data_child, 
                                                how="inner", 
                                                on=["timepoint", "replicate"],
                                                suffixes=["_parent", "_child"])
        parent_child_gene_dict[f"{parent_gene}-{child_gene}"] = [expression_data_parent_child["tpm_parent"].to_numpy().reshape(-1,1), expression_data_parent_child["tpm_child"].to_numpy().reshape(-1,1)]

In [7]:
network = get_network(parent_child_gene_dict)

In [8]:
network

,From,To,log_marginal_likelihood,lengthscale,variance,variance_n
0,G1,G3,20.711921,0.499999,0.002117,0.000001
0,G2,G3,-5.706432,0.200005,1391.531420,0.300000
